# 第 2 章: Apache Icebergの仕組みと機能

## 代表的なクエリのライフサイクル

### Spark Sessionと依存関係, utility関数の準備

In [38]:
import io
import struct
from pprint import pprint

import pandas as pd
from avro.datafile import DataFileReader
from avro.io import DatumReader

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

CATALOG = "spark_catalog"
CATALOG_URL = "http://server:8181/"
WAREHOUSE = "s3://amzn-s3-demo-bucket"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.1"

In [39]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.warehouse", WAREHOUSE)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", CATALOG)
        .getOrCreate()
)

In [40]:
%sql spark

In [41]:
def list_objects(bucket_name: str, prefix: str) -> None:
    """
    指定されたバケットとプレフィックスに基づいてオブジェクト一覧を表示
    """
    try:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                last_modified_str = obj['LastModified'].strftime("%Y-%m-%d %H:%M:%S")
                print(f"Key:          {obj['Key']}")
                print(f"LastModified: {last_modified_str}")
                print(f"Size:         {obj['Size']} bytes")
                print("-" * 40)
        else:
            print("オブジェクトが見つかりませんでした。")
            
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")

## boto3 クライアントの準備

以降の手順でminIO上のファイルを参照するため、boto3のS3クライアントを用意します。  
(手順をJupyter内で完結させるためにS3クライアントで操作しますが、適宜minIOの管理GUI`http://localhost:9001/browser`から参照しても構いません)  
  
＊実際にはS3を使用せず、ローカル上のminIOにアクセスしている点に注意してください。minIOはS3互換のオブジェクトストレージであるため、以下のようにboto3から操作できるのです) 
  
＊コード内にminIOのクレデンシャルをハードコードしていますが、**これを本番環境で真似しないでください。**  
これはローカルのハンズオン環境に接続するための簡易的な措置であり、コード内へのクレデンシャルのハードコードは非常にバッドなプラクティスです。本番向けには最低限環境変数に保持するか、適切なシークレットマネージャを使用してください。

In [42]:
import boto3
import json

# Minioサーバーの設定
minio_endpoint = 'http://minio:9000'
minio_access_key = 'admin'
minio_secret_key = 'password'
bucket_name = 'amzn-s3-demo-bucket'

# S3クライアントを作成
s3 = boto3.client('s3',
                  endpoint_url=minio_endpoint,
                  aws_access_key_id=minio_access_key,
                  aws_secret_access_key=minio_secret_key,
                  verify=False)

# バケットのリストを取得
response = s3.list_buckets()
print("Buckets:")
for bucket in response['Buckets']:
    print(f"- {bucket['Name']}")

Buckets:
- amzn-s3-demo-bucket


### ネームスペース作成

Icebergは「カタログ」「データベース（名前空間）」「テーブル」という論理コンポーネントを単位としてデータを管理します。<br/>
単一のカタログには複数のデータベースが存在し、各データベースには複数のテーブルが存在します。データベースは、名前空間(ネームスペース)と呼ばれることもあります。

In [43]:
%%sql

CREATE NAMESPACE IF NOT EXISTS demo

Running query in 'SparkSession'

""


### テーブル作成

続いて、先ほど作成したネームスペースに紐づくテーブルを作成してみましょう。`USING iceberg`の部分がポイントで、これによってIcebergテーブルを指定しています。

In [44]:
%%sql

-- 既にテーブルが存在する場合はDROP
-- テーブル作成時にCREATE OR REPLACEする形でもOK
DROP TABLE IF EXISTS demo.simple_table PURGE

Running query in 'SparkSession'

""


In [46]:
%%sql

CREATE TABLE IF NOT EXISTS demo.simple_table (
      id BIGINT,
      name STRING,
      score DOUBLE
    )
    USING iceberg
    LOCATION "s3://amzn-s3-demo-bucket/demo/simple_table"

Running query in 'SparkSession'

""


Icebergテーブルができました。`SHOW TABLES IN`で、ネームスペースのテーブルの一覧が確認できます。

In [48]:
%%sql

SHOW TABLES IN demo

Running query in 'SparkSession'

1 rows affected.

,namespace,tableName,isTemporary
0,demo,simple_table,False


テーブルのスキーマを確認したい時は`DESCRIBE`を使います。

Icebergでは、「テーブル識別子（Table Identifier）」を使用してテーブルを一意に識別します。<br/>
テーブル識別子（Table Identifier）は、`<カタログ名>.<データベース名>.<テーブル名>` の形式で指定します。<br/>
たとえば、カタログ名が`spark_catalog`、データベース名が`demo`、テーブル名が`simple_table`の場合、以下のようになります。<br/>
`spark_catalog.demo.simple_table`<br/><br/>
このノートブックでは、デフォルトで使用するカタログが指定済なので、カタログ名を省略して、`<データベース名>.<テーブル名>`(`demo.simple_table`)の形式で指定できます。

In [50]:
%%sql

DESCRIBE demo.simple_table

Running query in 'SparkSession'

3 rows affected.

,col_name,data_type,comment
0,id,bigint,None
1,name,string,None
2,score,double,None


`EXTENDED`オプションを付与することで、より詳細なスキーマ情報の確認も可能です。

In [8]:
%%sql

DESCRIBE EXTENDED demo.simple_table

Running query in 'SparkSession'

18 rows affected.

,col_name,data_type,comment
0,id,bigint,None
1,name,string,None
2,score,double,None
3,,,
4,# Metadata Columns,,
5,_spec_id,int,
6,_partition,struct<>,
7,_file,string,
8,_pos,bigint,
9,_deleted,boolean,


Icebergテーブルは、Icebergカタログが指し示すメタデータのロケーション情報を除く、ほぼ全てのメタデータ、実データをストレージに保持します。</br>
一般のユーザーがIcebergを利用する上で、ストレージ上のファイルを意識する必要は全くありませんが、中身の挙動を把握しておくことで、仕組みの理解が深まるほか、トラブルシュートにも役立ちます。

`CREATE TABLE`を通じて、Sどのようなデータが作られたかを確認してみましょう。</br>
テーブルのデータを配置先として指定したMinIOのパス（`s3://amzn-s3-demo-bucket/demo/simple_table`）を確認すると、`***.metadata.json`というJSONファイルが作られていることがわかります。これがメタデータファイルです。</br>
このJSONには、Icebergテーブルのある時点の状態に関するメタデータが保管されています。

In [51]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

Key:          demo/simple_table/metadata/00000-14072a13-f42c-497c-9f16-154bb732b0fe.metadata.json
LastModified: 2025-08-15 22:10:58
Size:         1158 bytes
----------------------------------------


メタデータファイルの中身を確認します。

In [10]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

最新のメタデータファイル: demo/simple_table/metadata/00000-d982f5e7-77d3-435f-b76a-0f0debcd60e8.metadata.json
{
  "format-version": 2,
  "table-uuid": "f5da540a-4f4b-48af-aa7e-31b1a10e8022",
  "location": "s3://amzn-s3-demo-bucket/demo/simple_table",
  "last-sequence-number": 0,
  "last-updated-ms": 1755256698623,
  "last-column-id": 3,
  "current-schema-id": 0,
  "schemas": [
    {
      "type": "struct",
      "schema-id": 0,
      "fields": [
        {
          "id": 1,
          "name": "id",
          "required": false,
          "type": "long"
        },
        {
          "id": 2,
          "name": "name",
          "required": false,
          "type": "string"
        },
        {
          "id": 3,
          "name": "score",
          "required": false,
          "type": "double"
        }
      ]
    }
  ],
  "default-spec-id": 0,
  "partition-specs": [
    {
      "spec-id": 0,
      "fields": []
    }
  ],
  "last-partition-id": 999,
  "default-sort-order-id": 0,
  "sort-orders": [
 

## INSERT

先ほど作成したテーブルにレコードを挿入してみましょう。

In [11]:
%%sql
    
INSERT INTO demo.simple_table VALUES
    (1, 'Alice', 85.5),
    (2, 'Bob', 90.0),
    (3, 'Charlie', 78.0);

Running query in 'SparkSession'

""


データの挿入が完了すると、Iceberg は新しいスナップショットを作成し、メタデータリスト、マニフェストファイル、データファイルを作成します。その上で、メタデータファイルを更新し、カタログにその情報を登録します。  
まずは、以下の SQL コマンドで挿入したデータが正しくテーブルに反映されていることを確認してください。

In [12]:
%%sql

SELECT * FROM demo.simple_table;

Running query in 'SparkSession'

3 rows affected.

,id,name,score
0,1,Alice,85.5
1,2,Bob,90.0
2,3,Charlie,78.0


次に、ストレージ上のファイルを観察してみましょう。`metadata`ディレクトリに新しいメタデータファイル、メタデータリスト、マニフェストファイルが作成されていることが分かります。それぞれを順に見ていきます。

In [13]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

Key:          demo/simple_table/metadata/00000-d982f5e7-77d3-435f-b76a-0f0debcd60e8.metadata.json
LastModified: 2025-08-15 11:18:18
Size:         1158 bytes
----------------------------------------
Key:          demo/simple_table/metadata/00001-1786454a-6079-464d-82dd-236aa67f522b.metadata.json
LastModified: 2025-08-15 11:18:21
Size:         2444 bytes
----------------------------------------
Key:          demo/simple_table/metadata/5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c-m0.avro
LastModified: 2025-08-15 11:18:21
Size:         6822 bytes
----------------------------------------
Key:          demo/simple_table/metadata/snap-4214241916683902793-1-5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c.avro
LastModified: 2025-08-15 11:18:21
Size:         4452 bytes
----------------------------------------


まずは最新のメタデータファイルを確認します。先ほどテーブルを作成した際に確認したメタデータファイルに比べて、INSERT時に作成されたsnapshotの情報が追加されていることが分かります。

In [14]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

最新のメタデータファイル: demo/simple_table/metadata/00001-1786454a-6079-464d-82dd-236aa67f522b.metadata.json
{
  "format-version": 2,
  "table-uuid": "f5da540a-4f4b-48af-aa7e-31b1a10e8022",
  "location": "s3://amzn-s3-demo-bucket/demo/simple_table",
  "last-sequence-number": 1,
  "last-updated-ms": 1755256701319,
  "last-column-id": 3,
  "current-schema-id": 0,
  "schemas": [
    {
      "type": "struct",
      "schema-id": 0,
      "fields": [
        {
          "id": 1,
          "name": "id",
          "required": false,
          "type": "long"
        },
        {
          "id": 2,
          "name": "name",
          "required": false,
          "type": "string"
        },
        {
          "id": 3,
          "name": "score",
          "required": false,
          "type": "double"
        }
      ]
    }
  ],
  "default-spec-id": 0,
  "partition-specs": [
    {
      "spec-id": 0,
      "fields": []
    }
  ],
  "last-partition-id": 999,
  "default-sort-order-id": 0,
  "sort-orders": [
 

それでは、スナップショットリストを確認します。

In [15]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    snap_avro_files = [
        obj for obj in response['Contents']
        if obj['Key'].startswith('demo/simple_table/metadata/snap-') and obj['Key'].endswith('.avro')
    ]
    if snap_avro_files:
        snap_avro_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_snap_key = snap_avro_files[0]['Key']
        print(f"最新の Snapshot AVRO ファイル: {latest_snap_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_snap_key)
        content = resp['Body'].read()

        bytes_stream = io.BytesIO(content)
        reader = DataFileReader(bytes_stream, DatumReader())

        print("AVRO ファイルのレコード:")
        with DataFileReader(bytes_stream, DatumReader()) as reader:
            for record in reader:
                pprint(record)
                print()
    else:
        print("snap-***.avro ファイルが見つかりませんでした。")
else:
    print("オブジェクトが見つかりませんでした。")

最新の Snapshot AVRO ファイル: demo/simple_table/metadata/snap-4214241916683902793-1-5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c.avro
AVRO ファイルのレコード:
{'added_files_count': 3,
 'added_rows_count': 3,
 'added_snapshot_id': 4214241916683902793,
 'content': 0,
 'deleted_files_count': 0,
 'deleted_rows_count': 0,
 'existing_files_count': 0,
 'existing_rows_count': 0,
 'key_metadata': None,
 'manifest_length': 6822,
 'manifest_path': 's3://amzn-s3-demo-bucket/demo/simple_table/metadata/5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c-m0.avro',
 'min_sequence_number': 1,
 'partition_spec_id': 0,
 'partitions': [],
 'sequence_number': 1}



続いて、マニフェストファイルを確認します。マニフェストファイルには、データファイルのパスや統計情報が含まれています。

In [16]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

def interpret_iceberg_bound_bytes(bound_value: bytes, column_id: int):
    """
    Icebergテーブルのマニフェストファイル内のバイナリを解釈する
    (coulumn_id ごとの型が事前に把握できている状況を想定したシンプルな実装)
      - col1 (id=1): BIGINT → 64bit 整数 (リトルエンディアン)
      - col2 (id=2): STRING → UTF-8 デコードを試行
      - col3 (id=3): DOUBLE → 64bit 浮動小数 (リトルエンディアン)
    """
    if column_id == 1:
        return struct.unpack('<q', bound_value)[0]
    elif column_id == 2:
        try:
            return bound_value.decode('utf-8')
        except UnicodeDecodeError:
            return bound_value.hex()
    elif column_id == 3:
        return struct.unpack('<d', bound_value)[0]
    else:
        return bound_value.hex()

def decode_bounds(bound_list: list):
    """
    lower_bounds / upper_bounds のような
    [{'key': <column_id>, 'value': <bytes>}, ...] という構造を
    interpret_iceberg_bound_bytes() でデコードする。
    """
    for item in bound_list:
        if isinstance(item['value'], bytes):
            col_id = item['key']
            item['value'] = interpret_iceberg_bound_bytes(item['value'], col_id)


if 'Contents' in response:
    # snap- を含まない .avro → マニフェストファイルを抽出
    manifest_avro_files = [
        obj for obj in response['Contents']
        if obj['Key'].endswith('.avro') and 'snap-' not in obj['Key']
    ]
    if manifest_avro_files:
        manifest_avro_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_manifest_key = manifest_avro_files[0]['Key']
        print(f"最新のマニフェストファイル: {latest_manifest_key}\n")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_manifest_key)
        content = resp['Body'].read()
        bytes_stream = io.BytesIO(content)

        print("マニフェストファイルのレコード:")
        with DataFileReader(bytes_stream, DatumReader()) as reader:
            for record in reader:
                df = record.get('data_file', {})
                if 'lower_bounds' in df:
                    decode_bounds(df['lower_bounds'])
                if 'upper_bounds' in df:
                    decode_bounds(df['upper_bounds'])

                pprint(record)
                print()
    else:
        print("マニフェストファイル (.avro かつ snap- 以外) が見つかりませんでした。")
else:
    print("オブジェクトが見つかりませんでした。")

最新のマニフェストファイル: demo/simple_table/metadata/5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c-m0.avro

マニフェストファイルのレコード:
{'data_file': {'column_sizes': [{'key': 1, 'value': 40},
                                {'key': 2, 'value': 41},
                                {'key': 3, 'value': 39}],
               'content': 0,
               'equality_ids': None,
               'file_format': 'PARQUET',
               'file_path': 's3://amzn-s3-demo-bucket/demo/simple_table/data/00000-216-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet',
               'file_size_in_bytes': 903,
               'key_metadata': None,
               'lower_bounds': [{'key': 1, 'value': 1},
                                {'key': 2, 'value': 'Alice'},
                                {'key': 3, 'value': 85.5}],
               'nan_value_counts': [{'key': 3, 'value': 0}],
               'null_value_counts': [{'key': 1, 'value': 0},
                                     {'key': 2, 'value': 0},
                                  

/home/jovyan/.local/lib/python3.11/site-packages/avro/schema.py:1233: IgnoredLogicalType: Unknown map, using array.
  warnings.warn(avro.errors.IgnoredLogicalType(f"Unknown {logical_type}, using {type_}."))


最後にデータファイルを観察します。Icebergのデータファイルの形式は Parquet, ORC, Avro から選択可能ですが、今回はデフォルトの Parquet でデータファイルが作成されていることがわかります。

In [17]:
prefix = 'demo/simple_table/data/'
list_objects(bucket_name, prefix)

Key:          demo/simple_table/data/00000-216-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         903 bytes
----------------------------------------
Key:          demo/simple_table/data/00001-217-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         890 bytes
----------------------------------------
Key:          demo/simple_table/data/00002-218-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         917 bytes
----------------------------------------


## UPDATE

次に、レコードをアップデートしてみましょう。以下の例では、id=2 のレコードの `score` を 100.0 に更新しています。  

In [18]:
%%sql
    
SELECT * FROM demo.simple_table

Running query in 'SparkSession'

3 rows affected.

,id,name,score
0,1,Alice,85.5
1,2,Bob,90.0
2,3,Charlie,78.0


In [19]:
%%sql
    
UPDATE demo.simple_table SET score = 100.0 WHERE id = 2;

Running query in 'SparkSession'

""


In [20]:
%%sql
    
SELECT * FROM demo.simple_table

Running query in 'SparkSession'

3 rows affected.

,id,name,score
0,2,Bob,100.0
1,1,Alice,85.5
2,3,Charlie,78.0


ここで改めて、ストレージ上のファイルを観察してみましょう。`metadata` ディレクトリにメタデータファイル、メタデータリスト、マニフェストファイルが作成されていることが分かります。

In [21]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

Key:          demo/simple_table/metadata/00000-d982f5e7-77d3-435f-b76a-0f0debcd60e8.metadata.json
LastModified: 2025-08-15 11:18:18
Size:         1158 bytes
----------------------------------------
Key:          demo/simple_table/metadata/00001-1786454a-6079-464d-82dd-236aa67f522b.metadata.json
LastModified: 2025-08-15 11:18:21
Size:         2444 bytes
----------------------------------------
Key:          demo/simple_table/metadata/00002-cf7f68f0-a6a1-4288-9a13-47ea3bf33aea.metadata.json
LastModified: 2025-08-15 11:18:29
Size:         3777 bytes
----------------------------------------
Key:          demo/simple_table/metadata/5e8fd1a6-c4c2-4f4a-aaee-aa100f1efe5c-m0.avro
LastModified: 2025-08-15 11:18:21
Size:         6822 bytes
----------------------------------------
Key:          demo/simple_table/metadata/b49800ed-3a46-4b6f-8b8e-a4e975a55505-m0.avro
LastModified: 2025-08-15 11:18:28
Size:         6838 bytes
----------------------------------------
Key:          demo/simple_table/me

In [22]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

最新のメタデータファイル: demo/simple_table/metadata/00002-cf7f68f0-a6a1-4288-9a13-47ea3bf33aea.metadata.json
{
  "format-version": 2,
  "table-uuid": "f5da540a-4f4b-48af-aa7e-31b1a10e8022",
  "location": "s3://amzn-s3-demo-bucket/demo/simple_table",
  "last-sequence-number": 2,
  "last-updated-ms": 1755256708912,
  "last-column-id": 3,
  "current-schema-id": 0,
  "schemas": [
    {
      "type": "struct",
      "schema-id": 0,
      "fields": [
        {
          "id": 1,
          "name": "id",
          "required": false,
          "type": "long"
        },
        {
          "id": 2,
          "name": "name",
          "required": false,
          "type": "string"
        },
        {
          "id": 3,
          "name": "score",
          "required": false,
          "type": "double"
        }
      ]
    }
  ],
  "default-spec-id": 0,
  "partition-specs": [
    {
      "spec-id": 0,
      "fields": []
    }
  ],
  "last-partition-id": 999,
  "default-sort-order-id": 0,
  "sort-orders": [
 

データファイルに関しても、更新前と比較して、新しいデータファイルが作成されていることが分かります。

In [23]:
prefix = 'demo/simple_table/data/'
list_objects(bucket_name, prefix)

Key:          demo/simple_table/data/00000-216-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         903 bytes
----------------------------------------
Key:          demo/simple_table/data/00000-231-68ccf8be-ddf5-4bc2-8d22-3d7f6cd8b0c4-0-00001.parquet
LastModified: 2025-08-15 11:18:28
Size:         913 bytes
----------------------------------------
Key:          demo/simple_table/data/00001-217-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         890 bytes
----------------------------------------
Key:          demo/simple_table/data/00002-218-66bbb96a-60ad-4743-b836-69f7ce1d06db-0-00001.parquet
LastModified: 2025-08-15 11:18:20
Size:         917 bytes
----------------------------------------


このように、Iceberg テーブルの更新時には新しいメタデータファイル、マニフェストリスト、マニフェストファイル、データファイルが作成されます。更新前のファイルはそのまま残り、メタデータファイル→マニフェストリスト→マニフェストファイル→データファイルのツリー構造のメタデータが更新されることで、データの変更を管理しています。

更新前のファイルが残っているので、過去のスナップショット ID を指定することで、過去のデータを参照することが可能です。これがタイムトラベルクエリです。  
スナップショット ID はメタデータファイルなどからも確認できますが、Icebergにはテーブルのスナップショットのログの参照に利用できる便利なメタデータテーブルが用意されています。

In [24]:
%%sql
    
SELECT committed_at,snapshot_id,operation  FROM demo.simple_table.snapshots ORDER BY committed_at;

Running query in 'SparkSession'

2 rows affected.

,committed_at,snapshot_id,operation
0,2025-08-15 11:18:21.319,4214241916683902793,append
1,2025-08-15 11:18:28.912,7836036387137093982,overwrite


この情報を元に、以下のようにスナップショット ID を指定して、過去のデータを参照できます。  

In [25]:
%%sql

-- INSERT 後、UPDATE 前のテーブルにクエリ
SELECT * FROM demo.simple_table VERSION AS OF <INSERT 後、UPDATE 前のスナップショットIDを指定>;

Running query in 'SparkSession'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:

[PARSE_SYNTAX_ERROR] Syntax error at or near '<'.(line 1, pos 46)

== SQL ==
SELECT * FROM demo.simple_table VERSION AS OF <INSERT 後、UPDATE 前のスナップショットIDを指定>;
----------------------------------------------^^^




## ビュー

以下のようにして、Icebergビューを作成できます。

In [34]:
%%sql
    
CREATE OR REPLACE VIEW demo.sample_view AS SELECT sum(score) FROM demo.simple_table

Running query in 'SparkSession'

""


In [35]:
%%sql

SELECT * FROM demo.sample_view

Running query in 'SparkSession'

1 rows affected.

,sum(score)
0,263.5


In [52]:
%%sql

DROP VIEW demo.sample_view

Running query in 'SparkSession'

""


In [ ]:
## テーブルの削除